# Immutable Data Structures in Python (and reactive programming ... maybe decorators too)

Alex Kavanagh (@ajkavananagh)

# What we'll try to cover

* Immutable Data Structures
* Decorators (because somebody asked ...)
* Reactive Programming
* Err, and why!

# Why Immutable Data Structures

* Contracts
* Reasoning
* Threading / parallelism




# Some basic immutable types


## tuple

In [66]:
x = (1, 2, 3)

print(x)

#x[1] = 4


(1, 2, 3)


(oops!)

## Strings (... really)

In [67]:
s = "abc"
#s[1] = 'd'

# Introducing collections.*

This is where all the goodness for making immutable structures live.

## collections.namedtuple

In [8]:
from collections import namedtuple

Complex = namedtuple('Complex', 'a b')

c = Complex(1,1)

print(c.a)

1


* It's like an immutable, lightweight, class, but you can't change it when it's done.

In [38]:
# c.a = 2

## collections.defaultdict

You should definitely explore `collections` as it contains some useful, er, collections.  for example, the `defaultdict`:

In [14]:
# have you ever wanted to do this (from the docs)

from collections import defaultdict

s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d = defaultdict(list)
for k, v in s:
    d[k].append(v)
dict(d.items())

{'blue': [2, 4], 'red': [1], 'yellow': [1, 3]}

# Let's make an immutable dictionary

We start at (Python3) `collections.abc.Mapping` (Python2, `collections.mapping`): [link](https://docs.python.org/3/library/collections.abc.html)

(Note: there is a `MutableMapping` too; but were not interested in that!)

We'll create an `AttrDict`: a dictionary than you can access by attribute.



## class AttrDict(...)

In [32]:
import collections.abc


class AttrDict(collections.abc.Mapping):
    
    def __init__(self, data):
        self._data = data.copy()
        
    def __getitem__(self, key):
        return self._data[key]
    
    __getattr__ = __getitem__
    
    def __setattr__(self, key, value):
        if key != '_data':
            raise TypeError("Can't set item in {}".format(self.__class__.__name__))
        super().__setattr__(key, value)
        
    def __len__(self):
        return len(self._data)
    
    def __iter__(self):
        return iter(self._data)
    
    def __repr__(self):
        return ("{}({{{}}})"
                .format(self.__class__.__name__,
                        ", ".join(["{}: {}".format(repr(k), repr(v))
                                   for k, v in self._data.items()])))

    def __str__(self):
        return "{{{}}}".format(", ".join(["{}: {}".format(k, v)
                                          for k, v in self._data.items()]))

In [37]:
a = AttrDict({'a': 1, 'b': 2, 'c': 3})
a.a
a.b
a.c
# a.b = 4

3

In [42]:
print(a)
a

{b: 2, c: 3, a: 1}


AttrDict({'b': 2, 'c': 3, 'a': 1})

# Decorators

Read my blog post: [Exploring Python Decorators](http://alex.kavanagh.name/2011/05/exploring-python-decorators/).

* http://alex.kavanagh.name/2011/05/exploring-python-decorators/

Simply: they are syntactic sugar for passing a function to another function when it is being defined:

In [58]:
from functools import wraps

def decorator(f):
    print("In decorator")
    @wraps(f)
    def dec(*args, **kwargs):
        # do something useful
        print("In dec")
        return f(*args, **kwargs)
    return dec

@decorator
def hello(string):
    print(string)

print("Before hello call")
hello("Python NE")

In decorator
Before hello call
In dec
Python NE


In [59]:
# This is roughy equivalent to:

def hello2(string):
    print("hello2", string)
    
hello2 = decorator(hello2)
hello2("there")

In decorator
In dec
hello2 there


## Decorator that takes arguments

In [60]:
def pdecorator(name):
    print("Before inner")
    def inner(f):
        print("In inner")
        def wrapped(*args, **kwargs):
            print("In wrapped")
            return f(name, *args, **kwargs)
        return wraps(f)(wrapped)
    return inner

@pdecorator('hello')
def hello3(string, string2='optional'):
    print(string, string2)
    
print("Before hello3")
    
hello3("again")

Before inner
In inner
Before hello3
In wrapped
hello again


In [61]:
# which is again, roughly equivalent to:

def hello4(greeting, name):
    print(greeting, name)
    
hello4 = pdecorator("Goodbye")(hello4)
hello4("all")

Before inner
In inner
In wrapped
Goodbye all


## Also a class way of doing a @decorator

In [65]:
class decorator():
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        print('Called {func} with args: {args}'.format(func=self.func.__name__,
                                                       args=args))
        return self.func(*args)

@decorator
def f1(x,y):
    return x,y

f1(1,2)

Called f1 with args: (1, 2)


(1, 2)

And also a way of of doing many other things.

# Reactive Programming

* Not to be confused with Reactor programming (which is something different)
* Think of a spreadsheet.

Reactive Systems are "Responsive, Resilient, Elastic and Message Driven" (Reactive Manifesto)

# What is Reactive Programming?

* Deals with flow of data
* Not necessarily 'functional programming' (although, **I** think it helps).
* Not 'event driven'

# Functional Reactive Programming

* No mutable state (... immutable data structures, anyone!)
* Easy to reaon about
* Easy to make smaller components

# Examples

* JS React
* ANY spreadsheet
* LINQ
* ReactiveX / RxPy

# Let's look at Rx

* This is from [their](RxPY/notebooks/Getting Started.ipynb) notebook.

(Switch notebooks now)

# What am I doing with it?

* It's still a work in progress
* Called `charms.declarative`
* Uses immutable data structures
* Uses a custom built reactive reactor to build chains of functions that produce result.
* Intend to use it for writing charms.
* Show some code!

References:

* https://www.slideshare.net/PeterLawrey/reactive-programming-with-examples
* https://en.wikipedia.org/wiki/Reactive_programming
* ReactiveX - API for async programming with observable streams: http://reactivex.io/
* ReactiveX / RxPy (Reactive Extensions for Python): https://github.com/ReactiveX/RxPY